# 🚀 NASA GCN Data Pipeline

Pipeline **Delta Live Tables (DLT)** para ingestão de eventos astronômicos da NASA GCN.

In [ ]:
import dlt
import sys
import struct
from datetime import datetime, timedelta

sys.path.append(spark.conf.get("bundle.sourcePath", "."))

from pyspark.sql.functions import (
    col, decode, split, current_timestamp, current_date, udf,
    from_json, regexp_extract, concat_ws, length, size, when, to_timestamp, lit,
    expr, trim, get_json_object, coalesce, regexp_replace, collect_list, count, max
)
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

from nasa_gcn.config import get_kafka_options
from nasa_gcn.utils import decode_utf8, clean_json_id
from nasa_gcn.schemas import CIRCULAR_SCHEMA
from nasa_gcn.binary_parser import parse_gcn_binary_packet, PARSED_BINARY_SCHEMA

In [ ]:
parse_binary_udf = udf(parse_gcn_binary_packet, PARSED_BINARY_SCHEMA)

## 🥉 Bronze Layer

In [ ]:
@dlt.table(name="gcn_raw", comment="Raw NASA GCN Kafka messages - Bronze layer",
           table_properties={"quality": "bronze"})
def gcn_raw():
    kafka_options = get_kafka_options()
    return (spark.readStream.format("kafka").options(**kafka_options).load()
        .select(
            col("key").cast("string").alias("message_key"),
            col("value"), col("topic"), col("partition"), col("offset"),
            col("timestamp").alias("kafka_timestamp"),
            current_timestamp().alias("ingestion_timestamp")))

## 🥈 Silver Layer

In [ ]:
@dlt.table(name="gcn_classic_text", comment="Classic text format GCN alerts optimized for RAG - Silver layer",
           table_properties={"quality": "silver"})
def gcn_classic_text():
    return (dlt.read_stream("gcn_raw")
        .filter(col("topic").startswith("gcn.classic.text."))
        .withColumn("text_decoded", decode_utf8())
        .select(
            col("message_key"),
            col("text_decoded").alias("message_text"),
            col("topic"), 
            split(col("topic"), r"\.").getItem(3).alias("event_type"),
            # Extracted RAG Fields
            regexp_extract(col("text_decoded"), r"TITLE:\s+(.*?)(?=\n)", 1).alias("title"),
            regexp_extract(col("text_decoded"), r"NOTICE_DATE:\s+(.*?)(?=\n)", 1).alias("notice_date"),
            regexp_extract(col("text_decoded"), r"NOTICE_TYPE:\s+(.*?)(?=\n)", 1).alias("notice_type"),
            # Document Text for RAG
            col("text_decoded").alias("document_text"),
            col("kafka_timestamp"), col("ingestion_timestamp"), col("partition"), col("offset"),
            current_timestamp().alias("silver_processed_timestamp"),
            current_date().alias("silver_processed_date")))

In [ ]:
@dlt.table(name="gcn_classic_voevent", comment="Classic VoEvent XML format GCN alerts optimized for RAG - Silver layer",
           table_properties={"quality": "silver"})
def gcn_classic_voevent():
    return (dlt.read_stream("gcn_raw")
        .filter(col("topic").startswith("gcn.classic.voevent."))
        .withColumn("xml_str", decode_utf8())
        .select(
            col("message_key"),
            col("xml_str").alias("voevent_xml"),
            col("topic"), 
            split(col("topic"), r"\.").getItem(3).alias("event_type"),
            # XPath Extractions for RAG
            expr("xpath_string(xml_str, '/*[local-name()=\"VOEvent\"]/@ivorn')").alias("ivorn"),
            expr("xpath_string(xml_str, '/*[local-name()=\"VOEvent\"]/@role')").alias("role"),
            expr("xpath_string(xml_str, '/*[local-name()=\"VOEvent\"]/*[local-name()=\"Who\"]/*[local-name()=\"Date\"]')").alias("date"),
            expr("xpath_string(xml_str, '/*[local-name()=\"VOEvent\"]/*[local-name()=\"Why\"]/*[local-name()=\"Inference\"]/*[local-name()=\"Concept\"]')").alias("concept"),
            # Document Text for RAG
            concat_ws(" | ",
                 concat_ws(": ", lit("ID"), expr("xpath_string(xml_str, '/*[local-name()=\"VOEvent\"]/@ivorn')")),
                 concat_ws(": ", lit("ROLE"), expr("xpath_string(xml_str, '/*[local-name()=\"VOEvent\"]/@role')")),
                 concat_ws(": ", lit("DATE"), expr("xpath_string(xml_str, '/*[local-name()=\"VOEvent\"]/*[local-name()=\"Who\"]/*[local-name()=\"Date\"]')")),
                 concat_ws(": ", lit("CONCEPT"), expr("xpath_string(xml_str, '/*[local-name()=\"VOEvent\"]/*[local-name()=\"Why\"]/*[local-name()=\"Inference\"]/*[local-name()=\"Concept\"]')")),
                 concat_ws(": ", lit("DESCRIPTION"), expr("xpath_string(xml_str, '/*[local-name()=\"VOEvent\"]/*[local-name()=\"How\"]/*[local-name()=\"Description\"]')"))
            ).alias("document_text"),
            col("kafka_timestamp"), col("ingestion_timestamp"), col("partition"), col("offset"),
            current_timestamp().alias("silver_processed_timestamp"),
            current_date().alias("silver_processed_date")))

In [ ]:
@dlt.table(name="gcn_classic_binary", comment="Classic binary format GCN alerts optimized for RAG - Silver layer",
           table_properties={"quality": "silver"})
def gcn_classic_binary():
    return (dlt.read_stream("gcn_raw")
        .filter(col("topic").startswith("gcn.classic.binary."))
        .withColumn("parsed", parse_binary_udf(col("value")))
        .select(
            col("message_key"),
            col("parsed.pkt_type"), col("parsed.pkt_type_name"), col("parsed.pkt_sernum"),
            col("parsed.trig_num"), col("parsed.burst_datetime"),
            col("parsed.burst_ra_deg"), col("parsed.burst_dec_deg"), col("parsed.burst_error_deg"),
            col("parsed.trigger_id"), col("parsed.parse_error"),
            col("topic"), split(col("topic"), r"\.").getItem(3).alias("event_type"),
            # Document Text for RAG
            concat_ws(" | ",
                concat_ws(": ", lit("TYPE"), col("parsed.pkt_type_name")),
                when(col("parsed.trig_num").isNotNull(), concat_ws(": ", lit("TRIG_NUM"), col("parsed.trig_num"))),
                when(col("parsed.burst_datetime").isNotNull(), concat_ws(": ", lit("DATE"), col("parsed.burst_datetime"))),
                when(col("parsed.burst_ra_deg").isNotNull(),
                    concat_ws(", ", concat_ws(": ", lit("RA"), col("parsed.burst_ra_deg")), 
                                    concat_ws(": ", lit("DEC"), col("parsed.burst_dec_deg"))))
            ).alias("document_text"),
            col("kafka_timestamp"), col("ingestion_timestamp"), col("partition"), col("offset"),
            current_timestamp().alias("silver_processed_timestamp"),
            current_date().alias("silver_processed_date"),
            col("value").alias("raw_binary")))

### GCN Notices - Otimizado para RAG

In [ ]:
@dlt.table(name="gcn_notices", comment="New JSON format GCN notices optimized for RAG - Silver layer",
           table_properties={"quality": "silver"})
def gcn_notices():
    """GCN Notices com campos comuns extraídos para RAG.
    
    Suporta múltiplas missões: IceCube, Super-Kamiokande, Einstein Probe, Fermi, Swift.
    Trata arrays JSON e campos nulos de diferentes schemas.
    """
    return (dlt.read_stream("gcn_raw")
        .filter(col("topic").startswith("gcn.notices."))
        .withColumn("json_str", decode_utf8())
        # Extrair campos raw
        .withColumn("_mission", get_json_object(col("json_str"), "$.mission"))
        .withColumn("_messenger", get_json_object(col("json_str"), "$.messenger"))
        .withColumn("_alert_type", get_json_object(col("json_str"), "$.alert_type"))
        .withColumn("_id_raw", coalesce(
            get_json_object(col("json_str"), "$.id"),
            get_json_object(col("json_str"), "$.event_name"),
            get_json_object(col("json_str"), "$.trigger_id")
        ))
        .select(
            col("message_key"),
            col("json_str").alias("notice_json"),
            col("topic"),
            # Missão do tópico
            split(col("topic"), r"\.").getItem(2).alias("mission"),
            # Campos extraídos com fallback para topic
            coalesce(col("_mission"), split(col("topic"), r"\.").getItem(2)).alias("mission_name"),
            get_json_object(col("json_str"), "$.instrument").alias("instrument"),
            coalesce(col("_messenger"), lit("Unknown")).alias("messenger"),
            # Notice ID limpo (sem colchetes de array)
            clean_json_id(col("_id_raw")).alias("notice_id"),
            get_json_object(col("json_str"), "$.pipeline").alias("pipeline"),
            coalesce(col("_alert_type"), lit("notice")).alias("alert_type"),
            get_json_object(col("json_str"), "$.alert_tense").alias("alert_tense"),
            # Timestamps
            get_json_object(col("json_str"), "$.trigger_time").alias("trigger_time"),
            get_json_object(col("json_str"), "$.alert_datetime").alias("alert_datetime"),
            # Coordenadas
            get_json_object(col("json_str"), "$.ra").cast("double").alias("ra"),
            get_json_object(col("json_str"), "$.dec").cast("double").alias("dec"),
            get_json_object(col("json_str"), "$.ra_dec_error").alias("ra_dec_error"),
            get_json_object(col("json_str"), "$.containment_probability").alias("containment_probability"),
            # Campos específicos de neutrinos
            get_json_object(col("json_str"), "$.n_events").alias("n_events"),
            get_json_object(col("json_str"), "$.nu_energy").alias("nu_energy"),
            get_json_object(col("json_str"), "$.p_astro").alias("p_astro"),
            get_json_object(col("json_str"), "$.luminosity_distance").alias("luminosity_distance"),
            # Metadados
            length(col("json_str")).alias("json_length"),
            # Document text para RAG (só inclui campos não-nulos)
            concat_ws(" | ",
                when(col("_mission").isNotNull(), concat_ws(": ", lit("MISSION"), col("_mission"))),
                when(col("_messenger").isNotNull(), concat_ws(": ", lit("MESSENGER"), col("_messenger"))),
                when(col("_alert_type").isNotNull(), concat_ws(": ", lit("TYPE"), col("_alert_type"))),
                when(col("_id_raw").isNotNull(), concat_ws(": ", lit("ID"), clean_json_id(col("_id_raw")))),
                when(get_json_object(col("json_str"), "$.ra").isNotNull(),
                    concat_ws(": ", lit("RA"), get_json_object(col("json_str"), "$.ra"))),
                when(get_json_object(col("json_str"), "$.dec").isNotNull(),
                    concat_ws(": ", lit("DEC"), get_json_object(col("json_str"), "$.dec")))
            ).alias("document_text"),
            col("kafka_timestamp"),
            col("ingestion_timestamp"),
            col("partition"),
            col("offset"),
            current_timestamp().alias("silver_processed_timestamp"),
            current_date().alias("silver_processed_date")))

### GCN Circulars - Otimizado para RAG

In [ ]:
@dlt.table(name="gcn_circulars", comment="GCN Circulars - astronomer reports optimized for RAG - Silver layer",
           table_properties={"quality": "silver"})
def gcn_circulars():
    return (dlt.read_stream("gcn_raw")
        .filter(col("topic") == "gcn.circulars")
        .withColumn("json_str", decode_utf8())
        .withColumn("parsed", from_json(col("json_str"), CIRCULAR_SCHEMA))
        .select(
            col("message_key"),
            col("json_str").alias("circular_json"),
            col("parsed.circularId").alias("circular_id"),
            col("parsed.eventId").alias("event_id"),
            col("parsed.subject").alias("subject"),
            col("parsed.body").alias("body"),
            col("parsed.submitter").alias("submitter"),
            trim(regexp_extract(col("parsed.submitter"), r"^([^<]+)", 1)).alias("submitter_name"),
            regexp_extract(col("parsed.submitter"), r"<([^>]+)>", 1).alias("submitter_email"),
            col("parsed.submittedHow").alias("submitted_how"),
            (col("parsed.createdOn") / 1000).cast("timestamp").alias("created_on"),
            regexp_extract(col("parsed.eventId"), r"^([A-Z]+)", 1).alias("event_type"),
            size(split(trim(col("parsed.body")), " ")).alias("word_count"),
            length(col("parsed.body")).alias("char_count"),
            concat_ws("\n",
                concat_ws(": ", lit("SUBJECT"), col("parsed.subject")),
                concat_ws(": ", lit("EVENT"), col("parsed.eventId")),
                concat_ws(": ", lit("AUTHOR"), trim(regexp_extract(col("parsed.submitter"), r"^([^<]+)", 1))),
                lit("---"),
                col("parsed.body")
            ).alias("document_text"),
            col("topic"),
            col("kafka_timestamp"),
            col("ingestion_timestamp"),
            col("partition"),
            col("offset"),
            current_timestamp().alias("silver_processed_timestamp"),
            current_date().alias("silver_processed_date")))

In [ ]:
@dlt.table(name="igwn_gwalert", comment="IGWN Gravitational Wave Alerts optimized for RAG - Silver layer",
           table_properties={"quality": "silver"})
def igwn_gwalert():
    return (dlt.read_stream("gcn_raw")
        .filter(col("topic") == "igwn.gwalert")
        .withColumn("json_str", decode_utf8())
        .withColumn("superevent_id", get_json_object(col("json_str"), "$.superevent_id"))
        .withColumn("alert_type", get_json_object(col("json_str"), "$.alert_type"))
        .withColumn("group", get_json_object(col("json_str"), "$.event.group"))
        .withColumn("pipeline", get_json_object(col("json_str"), "$.event.pipeline"))
        .withColumn("instruments", regexp_replace(get_json_object(col("json_str"), "$.event.instruments"), r"[\"\[\]]", ""))
        .withColumn("significant", get_json_object(col("json_str"), "$.event.significant"))
        .withColumn("gracedb_url", get_json_object(col("json_str"), "$.urls.gracedb"))
        .withColumn("prob_bns", get_json_object(col("json_str"), "$.event.classification.BNS"))
        .withColumn("prob_bbh", get_json_object(col("json_str"), "$.event.classification.BBH"))
        .withColumn("prob_nsbh", get_json_object(col("json_str"), "$.event.classification.NSBH"))
        .withColumn("prob_terrestrial", get_json_object(col("json_str"), "$.event.classification.Terrestrial"))
        .withColumn("prob_has_ns", get_json_object(col("json_str"), "$.event.properties.HasNS"))
        .withColumn("prob_has_remnant", get_json_object(col("json_str"), "$.event.properties.HasRemnant"))
        .withColumn("far", get_json_object(col("json_str"), "$.event.far"))
        .select(
            col("message_key"),
            col("json_str").alias("gwalert_json"),
            col("topic"),
            col("superevent_id"),
            col("alert_type"),
            get_json_object(col("json_str"), "$.time_created").alias("time_created"),
            col("gracedb_url"),
            col("significant"),
            col("group"),
            col("pipeline"),
            col("far"),
            col("instruments"),
            col("prob_bns"),
            col("prob_nsbh"),
            col("prob_bbh"),
            col("prob_terrestrial"),
            col("prob_has_ns"),
            col("prob_has_remnant"),
            # Document Text for RAG with null checks to avoid empty labels
            concat_ws(" | ",
                when(col("superevent_id").isNotNull(), concat_ws(": ", lit("ID"), col("superevent_id"))),
                when(col("alert_type").isNotNull(), concat_ws(": ", lit("TYPE"), col("alert_type"))),
                when(col("group").isNotNull(), concat_ws(": ", lit("GROUP"), col("group"))),
                when(col("pipeline").isNotNull(), concat_ws(": ", lit("PIPELINE"), col("pipeline"))),
                when(length(col("instruments")) > 0, concat_ws(": ", lit("INSTRUMENTS"), col("instruments"))),
                when(col("significant").isNotNull(), concat_ws(": ", lit("SIGNIFICANT"), col("significant"))),
                when(col("gracedb_url").isNotNull(), concat_ws(": ", lit("URL"), col("gracedb_url")))
            ).alias("document_text"),
            col("kafka_timestamp"), col("ingestion_timestamp"),
            col("partition"), col("offset"),
            current_timestamp().alias("silver_processed_timestamp"),
            current_date().alias("silver_processed_date")))

In [ ]:
@dlt.table(name="gcn_heartbeat", comment="GCN Heartbeat messages for monitoring - Silver layer",
           table_properties={"quality": "silver"})
def gcn_heartbeat():
    return (dlt.read_stream("gcn_raw")
        .filter(col("topic") == "gcn.heartbeat")
        .select(
            col("message_key"), decode_utf8().alias("heartbeat_json"),
            col("topic"), col("kafka_timestamp"), col("ingestion_timestamp"),
            col("partition"), col("offset"),
            current_timestamp().alias("silver_processed_timestamp"),
            current_date().alias("silver_processed_date")))

## 🥇 Gold Layer

In [ ]:
@dlt.table(name="gcn_events_summarized", comment="Aggregated events linking narrative (Circulars) with facts (Notices) - Gold Layer",
           table_properties={"quality": "gold"})
def gcn_events_summarized():
    # Leitura como Tabela (não stream) para permitir agregações globais
    circulars = dlt.read("gcn_circulars")
    gw_alerts = dlt.read("igwn_gwalert")

    # 1. Agregar Circulares por Event ID
    # Cria narrativa consolidada para RAG
    curr_circulars = (circulars
        .groupBy("event_id")
        .agg(
            count("circular_id").alias("circular_count"),
            concat_ws("\n\n---\n\n", collect_list("document_text")).alias("scientific_narrative"),
            max("created_on").alias("last_circular_date")
        )
        .filter(col("event_id").isNotNull())
    )

    # 2. Preparar GW Alerts para Join (chave superevent_id)
    curr_gw = (gw_alerts
        .withColumnRenamed("superevent_id", "event_id")
        .select("event_id", "alert_type", "time_created", "document_text")
        .withColumnRenamed("document_text", "alert_context")
        .withColumnRenamed("time_created", "alert_time")
    )

    # 3. Join Left (Baseado nas Circulares como fonte da verdade do evento nomeado)
    return (curr_circulars
        .join(curr_gw, "event_id", "left")
        .select(
            col("event_id"),
            col("circular_count"),
            col("last_circular_date"),
            coalesce(col("alert_time"), col("last_circular_date")).alias("event_time"),
            col("alert_type"),
            col("alert_context"),
            col("scientific_narrative"),
            current_timestamp().alias("gold_processed_timestamp")
        )
    )